# Regressão Linear Multipla Com Gradiente descendente

In [7]:
import numpy as np
import math
import time
import altair as alt
import numpy as np
from vega_datasets import data
import pandas as pd
import random
import altair as alt
from sklearn import linear_model

In [8]:
def compute_mse_vectorized(w,X,Y):
    res = Y - np.dot(X, w)
    totalError = np.dot(res.T, res)
    return totalError / float(len(Y))

Função para fazer uma atualização dos parâmetros no Gradiente Descendente:

$w^(t+1) = w^(t) − α ∇RSS(w^(t))$

Considerando $∇RSS(w^(t)) = −2HT (y−Hw)$ podemos substituír a função principal por:

$w^(t+1) = w^(t) − \alpha(−2HT (y−Hw))$

Nesse sentido abaixo é implementada a função que atualiza o valor de W. Onde:

learningRate = $\alpha$

H e w é o Hw da formúla

Y é o Y da formúla

In [9]:
def step_gradient_vectorized(w, H, Y, learningRate):
    gradient = np.multiply(-2, np.dot(H.T, Y - np.dot(H, w)))    
    new_w = np.subtract(w, np.multiply(learningRate, gradient))
    return [new_w, gradient]

In [10]:
def gradient_descent_runner_vectorized(starting_w, X,Y, learning_rate, epsilon):
    w = starting_w
    gradient = np.array([np.inf] * len(X[0]))
    i = 0
    while (np.linalg.norm(gradient) >= epsilon):
        w, gradient = step_gradient_vectorized(w, X, Y, learning_rate)
        if i % 50000 == 0:
            print("MSE na iteração {0} é de {1}".format(i,(compute_mse_vectorized(w, X, Y))[0][0]))
        i+= 1
    return w

#### Os dados utilizados
Os dados utilizados são as notas de alunos em disciplinas do primeiro período de Computação da UFCG.

In [11]:
my_data = pd.read_csv('sample_treino.csv') #read the data
my_data.head()

,Cálculo1,LPT,P1,IC,Cálculo2,cra
0,8.7,10.0,9.0,9.1,8.4,8.477647
1,7.0,7.0,7.7,7.0,6.2,6.851724
2,8.6,9.8,7.9,9.6,8.7,9.090588
3,7.8,8.3,6.8,8.2,8.0,7.283516
4,5.2,9.3,5.0,8.5,5.0,7.205747


#### Execução do Algoritmo
Abaixo é lido os dados com as notas e CRA dos alunos no primeiro período.

In [12]:
points = np.genfromtxt("sample_treino.csv", delimiter=",")

points = np.c_[np.ones(len(points)),points][1::]

#Features para predição (Cálculo1,LPT,P1,IC,Cálculo2)
X = points[:, :-1]

#CRA
Y = points[:, -1:]

init_w = np.zeros((len(points[0]) - 1,1))

learning_rate = 0.00001

epsilon = 0.04

print("Running...")
tic = time.time()

w = gradient_descent_runner_vectorized(init_w, X,Y, learning_rate, epsilon)

toc = time.time()
print("Gradiente descendente convergiu com w0 = {0}, w1 = {1}, w2 = {2}, w3 = {3}, w4 = {4}, w5 = {5}, error = {6}".format(w[0], w[1], w[2], w[3], w[4], w[5], compute_mse_vectorized(w,X,Y)))
print("Versão vetorizada rodou em: " + str(1000*(toc-tic)) + " ms")



Running...
MSE na iteração 0 é de 13.54011368883012
MSE na iteração 50000 é de 0.41731648956775663
MSE na iteração 100000 é de 0.413093359156704
MSE na iteração 150000 é de 0.4118531904032155
MSE na iteração 200000 é de 0.41148900119108417
MSE na iteração 250000 é de 0.41138205301821684
MSE na iteração 300000 é de 0.41135064650946057
Gradiente descendente convergiu com w0 = [1.70515148], w1 = [0.10362486], w2 = [0.04792676], w3 = [0.16402416], w4 = [0.38283179], w5 = [0.02067884], error = [[0.41134501]]
Versão vetorizada rodou em: 4558.002710342407 ms


### Sklearn vs Implementação
Abaixo são comparados os valores dos coeficientes obtidos com a execução biblioteca proposta com os obtidos com o algoritmo implementado neste Lab.

In [13]:
regr = linear_model.LinearRegression()
regr.fit(X[:, 1:], Y)
print('Coeficientes obtidos com a library sklearn: ', regr.coef_)
print('Intercept obetido com a library sklearn: ', regr.intercept_)
print('Coeficientes obtidos com o algoritmo implementado:', w[1], w[2], w[3], w[4], w[5])

Coeficientes obtidos com a library sklearn:  [[0.10304143 0.0464367  0.16409834 0.38117843 0.02027816]]
Intercept obetido com a library sklearn:  [1.73771151]
Coeficientes obtidos com o algoritmo implementado: [0.10362486] [0.04792676] [0.16402416] [0.38283179] [0.02067884]


Como é visto acima o algoritmo implementado obteve valores bastante similares ao produzidos pela library proposta no teste. Logo, podemos considerar que o a implementação está funcionando como o devido.